In [1]:
import  pandas as pd
from sklearn import metrics
import numpy as np
# from sklearn.metrics import r2_score, roc_curve, ndcg_score

def ndcg(y_pred, y_true):
    y_true_normalized = (y_true - y_true.mean()) / y_true.std()
    return metrics.ndcg_score(y_true_normalized.reshape(1, -1), y_pred.reshape(1, -1))

def classify(a, b, thld):
    pred = a
    tgt = b

    tp = fp = tn = fn = 1e-9
    pred_bin = []
    real_bin = []
    for i in range(len(tgt)):
        pred_i = pred[i]
        tgt_i = tgt[i]
        if pred_i > thld and tgt_i > thld:
            tp += 1
        if pred_i < thld and tgt_i > thld:
            fn += 1
        if pred_i > thld and tgt_i < thld:
            fp += 1
        if pred_i < thld and tgt_i < thld:
            tn += 1

        if pred_i >= thld:
            pred_bin.append(1)
        else:
            pred_bin.append(0)
        if tgt_i >= thld:
            real_bin.append(1)
        else:
            real_bin.append(0)
    # print(tp, tn ,fp ,fn)
    acc = (tp + tn) / (tp + tn + fp + fn)
    acc_t = tp / (tp + fp)
    acc_f = tn / (tn + fn)
    recall_t = tp / (tp + fn)
    recall_f = tn / (tn + fp)
    fpr, tpr, _ = metrics.roc_curve(real_bin, pred_bin)
    auc = metrics.auc(fpr, tpr)
    return acc, auc

def get_classfy_num(dataframe, index):
    sort_num = 100
    dataframe.sort_values(by = index, ascending=False, inplace = True)
    dataframe = dataframe.reset_index(drop = True)
    df_bright_sort = dataframe[:sort_num]
    # print(list(df_bright_sort["quantitative_function"]))
    num_1 = 0
    num_9 = 0
    num_11 = 0
    num_6 = 0
    for i in list(df_bright_sort["quantitative_function"]):
        if i > 1:
            num_1 += 1
        if i > 0.9:
            num_9 += 1
        if i > 1.1:
            num_11 += 1
        if i < 0.6:
            num_6 += 1
    # print("calssfy 1: " ,num_1, num_1 / sort_num)
    # print("calssfy 0.9: " ,num_9, num_9 / sort_num)
    # print("calssfy 1.1: " ,num_11, num_11 / sort_num)
    return num_1, num_9, num_11, num_6
    
def data_process_step1(input_path):
    step1_acc, step1_auc = 0, 0
    step1_classfy_1, step1_classfy_9, step1_classfy_11, step1_classfy_6 = 0, 0, 0, 0
    step1_acc_list, step1_auc_list = [], []
    step1_classfy_1_list, step1_classfy_9_list, step1_classfy_11_list, step1_classfy_6_list = [], [], [], []
    for seed in range(5):
        dataframe = pd.read_csv(f"{input_path}_{seed}.csv")
        df_bright = dataframe[dataframe["quantitative_function"] > 1]
        bright_num = len(list(df_bright["name"]))
        real_qfunc = np.array(list(dataframe["quantitative_function"]))
        predict_qfunc_1 = np.array(list(dataframe["predict_qfunc"]))
        acc_1, auc_1 = classify(predict_qfunc_1, real_qfunc, 1)
        step1_acc_list.append(acc_1)
        step1_acc += acc_1
        step1_auc_list.append(auc_1)
        step1_auc += auc_1
        step1_classfy_1_seed, step1_calssfy_9_seed, step1_calssfy_11_seed, step1_calssfy_6_seed = get_classfy_num(dataframe, "predict_qfunc")
        step1_classfy_1_list.append(step1_classfy_1_seed)
        step1_classfy_1 += step1_classfy_1_seed
        step1_classfy_9_list.append(step1_calssfy_9_seed)
        step1_classfy_9 += step1_calssfy_9_seed
        step1_classfy_11_list.append(step1_calssfy_11_seed)
        step1_classfy_11 += step1_calssfy_11_seed
        step1_classfy_6_list.append(step1_calssfy_6_seed)
        step1_classfy_6 += step1_calssfy_6_seed
    print("step1", step1_acc_list, step1_auc_list)
    print("step1 auc: ", step1_auc / 5)
    print("step1", step1_classfy_1_list, step1_classfy_9_list, step1_classfy_11_list)
    print("step1 classfy 1.0: ", step1_classfy_1 / 5, "step1 classfy 0.9: ", step1_classfy_9 / 5, "step1 classfy 1.1: ", step1_classfy_11 / 5, "step1 classfy < 0.6: ", step1_classfy_6 / 5)

In [5]:
model_name = "eUniRep"
vary_bright = 0
choose_step = "step1"
top_model = "nn"
if choose_step == "step1":
    input_path = f"/share/jake/hot_spot/data/test/method3/step_1/{top_model}/result_csv/{model_name}_random_1-2_gfp_SK_test_3"
    data_process_step1(input_path)
elif choose_step == "step2":
    if vary_bright:
        input_path = f"/share/jake/hot_spot/data/test/method3/step_1/{top_model}/result_csv/{model_name}_all_gfp_SK_test_3_step2_bright_continue_vary_bright"
    else:
        input_path = f"/share/jake/hot_spot/data/test/method3/step_1/{top_model}/result_csv/{model_name}_all_gfp_SK_test_3_step2_bright_continue"


step1 [0.7941152630298146, 0.823701061846373, 0.8670665477829724, 0.8581502796464754, 0.8547458863579946] [0.5685984646998341, 0.5494984978336711, 0.5205621088572838, 0.5267905210976859, 0.5399193087063242]
step1 auc:  0.5410737802389598
step1 [24, 25, 21, 26, 30] [71, 72, 90, 88, 80] [0, 0, 0, 2, 1]
step1 classfy 1.0:  25.2 step1 classfy 0.9:  80.2 step1 classfy 1.1:  0.6 step1 classfy < 0.6:  3.0


In [18]:

model_name = "UniRep"
vary_bright = 1
choose_step = "step_2"
top_model = "nn"
# seed = 4
# classfy_scor = 0.9
step1_acc, step1_auc, step2_acc, step2_auc = 0, 0, 0, 0
step1_classfy_1, step1_classfy_9, step1_classfy_11, step1_classfy_6 = 0, 0, 0, 0
step2_classfy_1, step2_classfy_9, step2_classfy_11, step2_classfy_6 = 0, 0, 0, 0
step1_acc_list, step1_auc_list, step2_acc_list, step2_auc_list = [], [], [], []
step1_classfy_1_list, step1_classfy_9_list, step1_classfy_11_list, step1_classfy_6_list = [], [], [], []
step2_classfy_1_list, step2_classfy_9_list, step2_classfy_11_list, step2_classfy_6_list = [], [], [], []
for seed in range(5):
    if vary_bright:
        input_path = f"/share/jake/hot_spot/data/test/method3/{choose_step}/{top_model}/result_csv/{model_name}_all_gfp_SK_test_3_step2_bright_continue_vary_bright_{seed}.csv"
    else:
        input_path = f"/share/jake/hot_spot/data/test/method3/{choose_step}/{top_model}/result_csv/{model_name}_all_gfp_SK_test_3_step2_bright_continue_{seed}.csv"
    dataframe = pd.read_csv(input_path)
    df_bright = dataframe[dataframe["quantitative_function"] > 1]
    bright_num = len(list(df_bright["name"]))
    real_qfunc = np.array(list(dataframe["quantitative_function"]))
    predict_qfunc_1 = np.array(list(dataframe["predict_qfunc"]))
    predict_qfunc_2 = np.array(list(dataframe["predict_qfunc_step2"]))
    acc_1, auc_1 = classify(predict_qfunc_1, real_qfunc, 1)
    step1_acc_list.append(acc_1)
    step1_acc += acc_1
    step1_auc_list.append(auc_1)
    step1_auc += auc_1
    # ndcg_1 = ndcg(predict_qfunc_1, real_qfunc)
    # print(acc_1, auc_1)
    acc_2, auc_2 = classify(predict_qfunc_2, real_qfunc, 1)
    step2_acc_list.append(acc_2)
    step2_acc += acc_2
    step2_auc_list.append(auc_2)
    step2_auc += auc_2
    # ndcg_2 = ndcg(predict_qfunc_2, real_qfunc)
    # print(acc_2, auc_2)
    step1_classfy_1_seed, step1_calssfy_9_seed, step1_calssfy_11_seed, step1_calssfy_6_seed = get_classfy_num(dataframe, "predict_qfunc")
    step1_classfy_1_list.append(step1_classfy_1_seed)
    step1_classfy_1 += step1_classfy_1_seed
    step1_classfy_9_list.append(step1_calssfy_9_seed)
    step1_classfy_9 += step1_calssfy_9_seed
    step1_classfy_11_list.append(step1_calssfy_11_seed)
    step1_classfy_11 += step1_calssfy_11_seed
    step1_classfy_6_list.append(step1_calssfy_6_seed)
    step1_classfy_6 += step1_calssfy_6_seed
    step2_classfy_1_seed, step2_calssfy_9_seed, step2_calssfy_11_seed, step2_calssfy_6_seed = get_classfy_num(dataframe, "predict_qfunc_step2")
    step2_classfy_1_list.append(step2_classfy_1_seed)
    step2_classfy_1 += step2_classfy_1_seed
    step2_classfy_9_list.append(step2_calssfy_9_seed)
    step2_classfy_9 += step2_calssfy_9_seed
    step2_classfy_11_list.append(step2_calssfy_11_seed)
    step2_classfy_11 += step2_calssfy_11_seed
    step2_classfy_6_list.append(step2_calssfy_6_seed)
    step2_classfy_6 += step2_calssfy_6_seed
print("step1", step1_acc_list, step1_auc_list)
print("step1 auc: ", step1_auc / 5)
print("step2", step2_acc_list, step2_auc_list)
print("step2 auc: ", step2_auc / 5)
print("step1", step1_classfy_1_list, step1_classfy_9_list, step1_classfy_11_list)
print("step2", step2_classfy_1_list, step2_classfy_9_list, step2_classfy_11_list)
print("step1 classfy 1.0: ", step1_classfy_1 / 5, "step1 classfy 0.9: ", step1_classfy_9 / 5, "step1 classfy 1.1: ", step1_classfy_11 / 5, "step1 classfy < 0.6: ", step1_classfy_6 / 5)
print("step2 classfy 1.0: ", step2_classfy_1 / 5, "step2 classfy 0.9: ", step2_classfy_9 / 5, "step2 classfy 1.1: ", step2_classfy_11 / 5, "step2 classfy < 0.6: ", step2_classfy_6 / 5)

step1 [0.8764271323025482, 0.8562793821347044, 0.8650100738741034, 0.5399597044995569, 0.562122229684185] [0.5, 0.5, 0.5, 0.5014630406208925, 0.49517744257369967]
step1 auc:  0.4993280966389184
step2 [0.8764271323025482, 0.8562793821347044, 0.8650100738741034, 0.8253861652106773, 0.8253861652106773] [0.5, 0.5, 0.5, 0.5, 0.5]
step2 auc:  0.5
step1 [8, 5, 7, 16, 15] [39, 59, 58, 81, 68] [0, 0, 0, 0, 1]
step2 [8, 20, 21, 11, 14] [59, 71, 69, 76, 74] [0, 0, 1, 0, 0]
step1 classfy 1.0:  10.2 step1 classfy 0.9:  61.0 step1 classfy 1.1:  0.2 step1 classfy < 0.6:  16.0
step2 classfy 1.0:  14.8 step2 classfy 0.9:  69.8 step2 classfy 1.1:  0.2 step2 classfy < 0.6:  10.8


In [10]:
sort_num = 100
dataframe.sort_values(by = "predict_qfunc", ascending=False, inplace = True)
dataframe = dataframe.reset_index(drop = True)
df_bright_sort = dataframe[:sort_num]
# print(list(df_bright_sort["quantitative_function"]))
num_1 = 0
num_9 = 0
num_11 = 0
for i in list(df_bright_sort["quantitative_function"]):
    if i > 1:
        num_1 += 1
    if i > 0.9:
        num_9 += 1
    if i > 1.1:
        num_11 += 1
print("calssfy 1: " ,num_1, num_1 / sort_num)
print("calssfy 0.9: " ,num_9, num_9 / sort_num)
print("calssfy 1.1: " ,num_11, num_11 / sort_num)

calssfy 1:  25 0.25
calssfy 0.9:  85 0.85
calssfy 1.1:  0 0.0


In [11]:
dataframe.sort_values(by = "predict_qfunc_step2", ascending=False, inplace = True)
dataframe = dataframe.reset_index(drop = True)
df_bright_sort = dataframe[:sort_num]
# print(list(df_bright_sort["quantitative_function"]))
# num = 0
# for i in list(df_bright_sort["quantitative_function"]):
#     if i > classfy_scor:
#         num += 1
# print(num, num / sort_num)
num_1 = 0
num_9 = 0
num_11 = 0
for i in list(df_bright_sort["quantitative_function"]):
    if i > 1:
        num_1 += 1
    if i > 0.9:
        num_9 += 1
    if i > 1.1:
        num_11 += 1
print("calssfy 1: " ,num_1, num_1 / sort_num)
print("calssfy 0.9: " ,num_9, num_9 / sort_num)
print("calssfy 1.1: " ,num_11, num_11 / sort_num)

calssfy 1:  35 0.35
calssfy 0.9:  84 0.84
calssfy 1.1:  1 0.01
